# Análisis Exploratorio de Datos | Lluvias en Australia

El dataset elegido contiene datos de aproximadamente 10 años de observaciones meteorológicas diarias de muchos lugares de Australia.

### Listado de variables y selección

#### VARIABLES DE ENTRADA

#### Utilizadas

* Date: fecha de observación.
* Location: nombre común de la ubicación de la estación meteorológica.
* MinTemp: temperatura mínima en grados centígrados.
* MaxTemp: temperatura máxima en grados centígrados.
* Rainfall: la cantidad de lluvia registrada para el día en mm.
* Evaporation: la llamada evaporación de la bandeja de clase A (mm) en las 24 horas anteriores a las 9 am.
* Sunshine: número de horas de sol brillante en el día.
* Humidity9am: humedad (porcentaje) a las 9 am.
* Humidity3pm: humedad (porcentaje) a las 3 pm.
* Pressure9am: presión atmosférica (hpa) reducida al nivel medio del mar a las 9 am.
* Pressure3pm: presión atmosférica (hpa) reducida al nivel medio del mar a las 3 pm.
* Cloud9am: fracción del cielo oscurecida por las nubes a las 9 am. Esto se mide en “octas”, que son una unidad de octavos. Registra cuántos octavos del cielo están oscurecidos por las nubes. Una medida 0 indica cielo completamente despejado, mientras que un 8 indica que está completamente nublado.
* Cloud3pm: fracción de cielo oscurecida por nubes (en “octas”: octavos) a las 3 pm.
* RainToday: booleano. 1 si la precipitación (mm) en las 24 horas anteriores a las 9 am excede 1 mm, de lo contrario 0.

#### No utilizadas

Descartamos las siguientes variables porque no consideramos que sean determinantes para que llueva:
* WindGustDir: dirección de la ráfaga de viento más fuerte en las 24 horas anteriores a la medianoche.
* WindGustSpeed: velocidad (km/h) de la ráfaga de viento más fuerte en las 24 horas anteriores a la medianoche.
* WindDir9am: dirección del viento a las 9 am.
* WindDir3pm: dirección del viento a las 3 pm.
* WindSpeed9am: velocidad del viento (km/h) promediada durante 10 minutos antes de las 9 am.
* WindSpeed3pm: velocidad del viento (km/h) promediada durante 10 minutos antes de las 3 pm.

Decidimos no tenerlas en cuenta porque ya consideramos la temperatura máxima y mínima:
* Temp9am: temperatura (grados centígrados) a las 9 am.
* Temp3pm: temperatura (grados centígrados) a las 3 pm.

#### VARIABLE DE SALIDA

* RainTomorrow: si, dados los datos de un día, lloverá o no al día siguiente. Se considera que sí llovió, a partir de 1mm. Valores posibles: Yes, No.

### Descripción general

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
dataset = pd.read_csv('weatherAUS.csv')
del dataset['WindGustDir']
del dataset['WindGustSpeed']
del dataset['WindDir9am']
del dataset['WindDir3pm']
del dataset['WindSpeed9am']
del dataset['WindSpeed3pm']
del dataset['Temp9am']
del dataset['Temp3pm']
dataset.sample(5)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,RainToday,RainTomorrow
66305,2015-01-13,MelbourneAirport,19.7,25.5,0.0,9.4,3.1,69.0,82.0,1003.8,997.7,8.0,7.0,No,Yes
46038,2009-01-25,Canberra,15.1,27.7,0.0,11.2,7.9,75.0,51.0,1018.7,1016.5,8.0,0.0,No,No
47405,2012-11-22,Canberra,12.6,22.3,0.0,7.2,10.0,72.0,46.0,1016.0,1014.5,7.0,NaN,No,No
81166,2009-06-18,Dartmoor,1.7,14.8,0.0,1.0,8.7,99.0,65.0,1029.3,1026.7,NaN,NaN,No,No
135920,2017-01-26,Launceston,11.2,26.6,0.0,NaN,NaN,68.0,39.0,NaN,NaN,NaN,NaN,No,No


In [15]:
dataset.isnull().sum()

Date                0
Location            0
MinTemp          1485
MaxTemp          1261
Rainfall         3261
Evaporation     62790
Sunshine        69835
Humidity9am      2654
Humidity3pm      4507
Pressure9am     15065
Pressure3pm     15028
Cloud9am        55888
Cloud3pm        59358
RainToday        3261
RainTomorrow     3267
dtype: int64

No detectamos diferencias en las unidades de medida de los datos de entrada, por lo que no consideramos que sea necesario realizar transformaciones. En cuanto al target o variable de salida (RainTomorrow), decidimos eliminar las filas que tienen valores en nulo para que únicamente queden registros con valores "Yes" o "No".

In [22]:
dataset.shape

(145460, 15)

In [28]:
#dataset2 = dataset.drop(dataset[dataset['RainTomorrow']==NA].index)
dataset.drop(dataset[dataset.RainTomorrow.isnull()].index, inplace = True)
dataset.sample(5)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,RainToday,RainTomorrow
69027,2013-09-29,Melbourne,7.6,21.4,NaN,4.2,9.3,49.0,42.0,1014.9,1011.0,NaN,NaN,NaN,Yes
139222,2008-10-20,Darwin,25.2,37.0,0.0,10.4,11.3,64.0,51.0,1012.7,1007.5,1.0,1.0,No,No
10758,2013-11-25,CoffsHarbour,14.9,24.7,0.2,4.8,11.5,63.0,61.0,1007.5,1005.6,1.0,5.0,No,Yes
136244,2009-05-23,AliceSprings,8.3,28.6,0.0,3.2,10.4,41.0,17.0,1013.7,1008.7,0.0,0.0,No,No
98766,2015-06-10,Adelaide,8.7,13.9,0.0,1.6,NaN,68.0,45.0,1036.7,1035.0,NaN,NaN,No,No


In [29]:
dataset.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm
count,141556.000000,141871.000000,140787.000000,81350.000000,74377.000000,140419.000000,138583.000000,128179.000000,128212.000000,88536.000000,85099.000000
mean,12.186400,23.226784,2.349974,5.469824,7.624853,68.843810,51.482606,1017.653758,1015.258204,4.437189,4.503167
std,6.403283,7.117618,8.465173,4.188537,3.781525,19.051293,20.797772,7.105476,7.036677,2.887016,2.720633
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,0.000000,0.000000,980.500000,977.100000,0.000000,0.000000
25%,7.600000,17.900000,0.000000,2.600000,4.900000,57.000000,37.000000,1012.900000,1010.400000,1.000000,2.000000
50%,12.000000,22.600000,0.000000,4.800000,8.500000,70.000000,52.000000,1017.600000,1015.200000,5.000000,5.000000
75%,16.800000,28.200000,0.800000,7.400000,10.600000,83.000000,66.000000,1022.400000,1020.000000,7.000000,7.000000
max,33.900000,48.100000,371.000000,145.000000,14.500000,100.000000,100.000000,1041.000000,1039.600000,9.000000,9.000000


In [30]:
dataset.isnull().sum()

Date                0
Location            0
MinTemp           637
MaxTemp           322
Rainfall         1406
Evaporation     60843
Sunshine        67816
Humidity9am      1774
Humidity3pm      3610
Pressure9am     14014
Pressure3pm     13981
Cloud9am        53657
Cloud3pm        57094
RainToday        1406
RainTomorrow        0
dtype: int64

In [31]:
dataset.shape

(142193, 15)